# 演習2：Steane符号と論理ビット操作

演習1では単一の量子ビットに対して、符号化した量子ビットに対して、ノイズを乗せる実験を行った。  
この演習では、符号化された論理ビットに対して各種ゲート操作に対応する操作を行い、ベル状態を作成してみる。

ここでは、Shorの符号ではなく、Steaneの符号を用いる。

# ライブラリのインポート

In [ ]:
%pip install qiskit qiskit-aer
%pip install pylatexenc

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit_aer.noise import pauli_error


# エラーチャネルの作成

課題1と同様にエラーを起こす部分を用意します。

ここでは先ほどよりも少し高めのエラー率にしています。  
理由としては、ここでは17量子ビットを使用する長い回路を利用するため、shot数を多く確保すると実行時間伸びてしまいます。  
そのため、shot数を抑えたいのですが、低めのエラー率だと、エラーが出て欲しい場面で観測されない可能性があるため、エラー率を高めています。

In [ ]:
n_shots = 1000  # シミュレーターでのサンプリング回数
backend_sim = AerSimulator()  # シミュレーターの用意
p_error = 0.1  # エラー確率


In [ ]:
def make_bitphase_error_channel(p_error, print_flag=True):
    bit_flip = pauli_error([("X", p_error), ("I", 1 - p_error)])
    phase_flip = pauli_error([("Z", p_error), ("I", 1 - p_error)])

    bitphase_flip = bit_flip.compose(phase_flip)

    if print_flag:
        print(bitphase_flip)
    return bitphase_flip


In [ ]:
bitphase_flip = make_bitphase_error_channel(p_error)


まずは、符号化をしないままのベル状態に1量子ビットだけノイズをかけてみます。

In [ ]:
# エラーありの回路
n_qubits = 2
circ_noise = QuantumCircuit(n_qubits, 2)

circ_noise.h(0)
circ_noise.cx(0, 1)

circ_noise.append(bitphase_flip, [0])

circ_noise.measure([0, 1], [0, 1])
circ_noise.draw("mpl")


In [ ]:
result_noise = backend_sim.run(circ_noise, shots=n_shots).result()
plot_histogram(result_noise.get_counts(0))


本来出現しないはずの01と11がそれぞれ約5％の確率で出現しています。（合計10％のエラー率）

# Steane符号

まずは1ビットの情報をSteane符号で符号化して、挙動を確認してみます。

Xゲートで反転させただけの状態を用意し、符号化します。

詳細については、以下のwebサイトが参考になります。
- https://intra.ece.ucr.edu/~korotkov/courses/EE214-QC/QC-7-error-correction.pdf

In [ ]:
def steane_code(noise_channel=[], p_error=0.1):
    # noise_channel : ノイズをかけたいチャネル(int)を入れたリスト
    # p_error : エラーの発生確率

    # エラーの定義
    bitphase_flip = make_bitphase_error_channel(p_error, print_flag=False)

    # 回路の記述
    n_qubits = 7 + 6
    circ = QuantumCircuit(n_qubits, 1)

    # データビットは3ビット目
    circ.x(3)

    circ.barrier()

    # 符号化
    circ.h(0)
    circ.h(1)
    circ.h(2)

    circ.cx(3, 4)
    circ.cx(3, 5)

    circ.cx(2, 3)
    circ.cx(2, 4)
    circ.cx(2, 6)

    circ.cx(1, 3)
    circ.cx(1, 5)
    circ.cx(1, 6)

    circ.cx(0, 4)
    circ.cx(0, 5)
    circ.cx(0, 6)

    circ.barrier()

    # エラーが発生する部分
    for i in noise_channel:
        assert (0 <= i) and (i < 13)
        circ.append(bitphase_flip, [i])

    circ.barrier()

    # エラー訂正

    # アンシラに情報を送る
    for i in range(6):
        circ.reset(i + 7)
        circ.h(i + 7)

    circ.cz(7, 0)
    circ.cz(7, 4)
    circ.cz(7, 5)
    circ.cz(7, 6)

    circ.cz(8, 1)
    circ.cz(8, 3)
    circ.cz(8, 5)
    circ.cz(8, 6)

    circ.cz(9, 2)
    circ.cz(9, 3)
    circ.cz(9, 4)
    circ.cz(9, 6)

    circ.cx(10, 0)
    circ.cx(10, 4)
    circ.cx(10, 5)
    circ.cx(10, 6)

    circ.cx(11, 1)
    circ.cx(11, 3)
    circ.cx(11, 5)
    circ.cx(11, 6)

    circ.cx(12, 2)
    circ.cx(12, 3)
    circ.cx(12, 4)
    circ.cx(12, 6)

    for i in range(6):
        circ.h(i + 7)

    circ.barrier()

    # 位相反転
    circ.x(11)
    circ.x(12)
    circ.h(0)
    circ.mcx([10, 11, 12], 0)
    circ.h(0)
    circ.x(11)
    circ.x(12)

    circ.x(10)
    circ.x(12)
    circ.h(1)
    circ.mcx([10, 11, 12], 1)
    circ.h(1)
    circ.x(10)
    circ.x(12)

    circ.x(10)
    circ.x(11)
    circ.h(2)
    circ.mcx([10, 11, 12], 2)
    circ.h(2)
    circ.x(10)
    circ.x(11)

    circ.x(10)
    circ.h(3)
    circ.mcx([10, 11, 12], 3)
    circ.h(3)
    circ.x(10)

    circ.x(11)
    circ.h(4)
    circ.mcx([10, 11, 12], 4)
    circ.h(4)
    circ.x(11)

    circ.x(12)
    circ.h(5)
    circ.mcx([10, 11, 12], 5)
    circ.h(5)
    circ.x(12)

    circ.h(6)
    circ.mcx([10, 11, 12], 6)
    circ.h(6)

    # ビット反転
    circ.x(8)
    circ.x(9)
    circ.mcx([7, 8, 9], 0)
    circ.x(8)
    circ.x(9)

    circ.x(7)
    circ.x(9)
    circ.mcx([7, 8, 9], 1)
    circ.x(7)
    circ.x(9)

    circ.x(7)
    circ.x(8)
    circ.mcx([7, 8, 9], 2)
    circ.x(7)
    circ.x(8)

    circ.x(7)
    circ.mcx([7, 8, 9], 3)
    circ.x(7)

    circ.x(8)
    circ.mcx([7, 8, 9], 4)
    circ.x(8)

    circ.x(9)
    circ.mcx([7, 8, 9], 5)
    circ.x(9)

    circ.mcx([7, 8, 9], 6)

    circ.barrier()

    # 復号
    circ.cx(0, 4)
    circ.cx(0, 5)
    circ.cx(0, 6)

    circ.cx(1, 3)
    circ.cx(1, 5)
    circ.cx(1, 6)

    circ.cx(2, 3)
    circ.cx(2, 4)
    circ.cx(2, 6)

    circ.cx(3, 4)
    circ.cx(3, 5)

    circ.h(0)
    circ.h(1)
    circ.h(2)

    circ.measure([3], [0])

    return circ


In [ ]:
circ = steane_code()


In [ ]:
circ.draw("mpl")


In [ ]:
result_ideal = backend_sim.run(circ, shots=n_shots).result()
plot_histogram(result_ideal.get_counts(0))


# 誤り訂正

Steane符号を使って、1量子ビットまでの誤りが訂正できることを確認します。

In [ ]:
circ = steane_code(noise_channel=[0])


In [ ]:
result_noise = backend_sim.run(circ, shots=n_shots).result()
plot_histogram(result_noise.get_counts(0))


In [ ]:
circ = steane_code(noise_channel=[3])


In [ ]:
result_noise = backend_sim.run(circ, shots=n_shots).result()
plot_histogram(result_noise.get_counts(0))


また、2量子ビット以上の場合は訂正できないことを確認してみます。

In [ ]:
circ = steane_code(noise_channel=[0, 3])


In [ ]:
result_noise = backend_sim.run(circ, shots=n_shots).result()
plot_histogram(result_noise.get_counts(0))


----

# 論理ビット操作によるベル状態の作成

今度はベル状態の作成を試すのですが、事前にベル状態を作成したものを符号化するのではなく、符号化された論理量子ビットに対して、論理ビット操作を加えて、ベル状態を作成します。

先ほどの例では、補助量子ビットを6量子ビット使っていますが、量子ビット数を節約するため、リセットしながら3量子ビットを使い回して実装します。

In [ ]:
def steane_code_bell_state(noise_channel=[], p_error=0.1):
    # noise_channel : ノイズをかけたいチャネル(int)を入れたリスト
    # p_error : エラーの発生確率

    # エラーの定義
    bitphase_flip = make_bitphase_error_channel(p_error, print_flag=False)

    # 回路の記述
    n_qubits = 14 + 3
    circ = QuantumCircuit(n_qubits, 2)

    # オリジナルのデータは3ビット目、10ビット目
    # アンシラは14〜17ビット目

    circ.barrier()

    # 符号化
    for i in range(2):
        circ.h(0 + i * 7)
        circ.h(1 + i * 7)
        circ.h(2 + i * 7)

        circ.cx(3 + i * 7, 4 + i * 7)
        circ.cx(3 + i * 7, 5 + i * 7)

        circ.cx(2 + i * 7, 3 + i * 7)
        circ.cx(2 + i * 7, 4 + i * 7)
        circ.cx(2 + i * 7, 6 + i * 7)

        circ.cx(1 + i * 7, 3 + i * 7)
        circ.cx(1 + i * 7, 5 + i * 7)
        circ.cx(1 + i * 7, 6 + i * 7)

        circ.cx(0 + i * 7, 4 + i * 7)
        circ.cx(0 + i * 7, 5 + i * 7)
        circ.cx(0 + i * 7, 6 + i * 7)

    circ.barrier()

    # エラーチャネル
    # ここで、論理アダマールゲートと論理CNOTゲートをかける
    for i in range(7):
        circ.h(i)

    for i in range(7):
        circ.cx(i, i + 7)

    # エラーは以下で発生させる
    for i in noise_channel:
        assert (0 <= i) and (i < 17)
        circ.append(bitphase_flip, [i])

    # エラー訂正

    for i in range(2):  # 論理ビット数に対応するループ
        circ.barrier()
        for j in range(3):  # アンシラの初期化（同じものを使い回す）
            circ.reset(j + 14)
            circ.h(j + 14)

        # アンシラに情報を送る
        circ.cz(14, 0 + i * 7)
        circ.cz(14, 4 + i * 7)
        circ.cz(14, 5 + i * 7)
        circ.cz(14, 6 + i * 7)

        circ.cz(15, 1 + i * 7)
        circ.cz(15, 3 + i * 7)
        circ.cz(15, 5 + i * 7)
        circ.cz(15, 6 + i * 7)

        circ.cz(16, 2 + i * 7)
        circ.cz(16, 3 + i * 7)
        circ.cz(16, 4 + i * 7)
        circ.cz(16, 6 + i * 7)

        for j in range(3):
            circ.h(j + 14)

        circ.barrier()

        # ビット反転
        circ.x(15)
        circ.x(16)
        circ.mcx([14, 15, 16], 0 + i * 7)
        circ.x(15)
        circ.x(16)

        circ.x(14)
        circ.x(16)
        circ.mcx([14, 15, 16], 1 + i * 7)
        circ.x(14)
        circ.x(16)

        circ.x(14)
        circ.x(15)
        circ.mcx([14, 15, 16], 2 + i * 7)
        circ.x(14)
        circ.x(15)

        circ.x(14)
        circ.mcx([14, 15, 16], 3 + i * 7)
        circ.x(14)

        circ.x(15)
        circ.mcx([14, 15, 16], 4 + i * 7)
        circ.x(15)

        circ.x(16)
        circ.mcx([14, 15, 16], 5 + i * 7)
        circ.x(16)

        circ.mcx([14, 15, 16], 6 + i * 7)

        # アンシラをリセットして使い回す
        circ.barrier()
        for j in range(3):
            circ.reset(j + 14)
            circ.h(j + 14)

        circ.cx(14, 0 + i * 7)
        circ.cx(14, 4 + i * 7)
        circ.cx(14, 5 + i * 7)
        circ.cx(14, 6 + i * 7)

        circ.cx(15, 1 + i * 7)
        circ.cx(15, 3 + i * 7)
        circ.cx(15, 5 + i * 7)
        circ.cx(15, 6 + i * 7)

        circ.cx(16, 2 + i * 7)
        circ.cx(16, 3 + i * 7)
        circ.cx(16, 4 + i * 7)
        circ.cx(16, 6 + i * 7)

        for j in range(3):
            circ.h(j + 14)

        circ.barrier()

        # 位相反転
        circ.x(15)
        circ.x(16)
        circ.h(0 + i * 7)
        circ.mcx([14, 15, 16], 0 + i * 7)
        circ.h(0 + i * 7)
        circ.x(15)
        circ.x(16)

        circ.x(14)
        circ.x(16)
        circ.h(1 + i * 7)
        circ.mcx([14, 15, 16], 1 + i * 7)
        circ.h(1 + i * 7)
        circ.x(14)
        circ.x(16)

        circ.x(14)
        circ.x(15)
        circ.h(2 + i * 7)
        circ.mcx([14, 15, 16], 2 + i * 7)
        circ.h(2 + i * 7)
        circ.x(14)
        circ.x(15)

        circ.x(14)
        circ.h(3 + i * 7)
        circ.mcx([14, 15, 16], 3 + i * 7)
        circ.h(3 + i * 7)
        circ.x(14)

        circ.x(15)
        circ.h(4 + i * 7)
        circ.mcx([14, 15, 16], 4 + i * 7)
        circ.h(4 + i * 7)
        circ.x(15)

        circ.x(16)
        circ.h(5 + i * 7)
        circ.mcx([14, 15, 16], 5 + i * 7)
        circ.h(5 + i * 7)
        circ.x(16)

        circ.h(6 + i * 7)
        circ.mcx([14, 15, 16], 6 + i * 7)
        circ.h(6 + i * 7)

    circ.barrier()

    # 復号
    for i in range(2):
        circ.cx(0 + i * 7, 4 + i * 7)
        circ.cx(0 + i * 7, 5 + i * 7)
        circ.cx(0 + i * 7, 6 + i * 7)

        circ.cx(1 + i * 7, 3 + i * 7)
        circ.cx(1 + i * 7, 5 + i * 7)
        circ.cx(1 + i * 7, 6 + i * 7)

        circ.cx(2 + i * 7, 3 + i * 7)
        circ.cx(2 + i * 7, 4 + i * 7)
        circ.cx(2 + i * 7, 6 + i * 7)

        circ.cx(3 + i * 7, 4 + i * 7)
        circ.cx(3 + i * 7, 5 + i * 7)

        circ.h(0 + i * 7)
        circ.h(1 + i * 7)
        circ.h(2 + i * 7)

    # 測定
    circ.measure([3, 10], [0, 1])

    return circ


In [ ]:
circ = steane_code_bell_state()


以下のコードで回路を描画できますが、非常に巨大な回路になります。

In [ ]:
circ.draw("mpl")


まずは誤りをかけない状態で、ベル状態が作れていることを確認します。

In [ ]:
%%time
# 実行に4分程度かかります
result_ideal = backend_sim.run(circ, shots=n_shots).result()
plot_histogram(result_ideal.get_counts(0))

# 課題1

この回路にエラーを加えて、訂正できることを確認してみましょう。

# 課題2

誤りの発生確率を変化させながら、実際に誤りが訂正できる確率を確認してみましょう。